In [ ]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/mw/work/

In [ ]:
# 查看当前kernel下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [1]:
import pandas as pd
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import lightgbm as lgb
import catboost as cbt
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold,StratifiedKFold

In [2]:
seed =2021

In [36]:
# df_train = pd.read_csv('/home/mw/input/tianchi6504/model_sample.csv')
df_train = pd.read_csv('/home/mw/work/features.csv',index_col =0)
df_train.reset_index()

df_train.head()

,user_id,y,x_001,x_002,x_003,x_004,x_005,x_006,x_007,x_008,...,x_184/x_179,x_185/x_180,x_189/x_188,x_191/x_190,x_193/x_192,x_195/x_194,x_197/x_196,x_199/x_198,x_196/x_188,x_192/x_188
0,A00002,0,0.0,32.0,0,0,0,0,0,0,...,0.25000,1000.000000,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
1,A00005,0,0.0,29.0,0,0,0,0,0,0,...,-999.00000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,A00006,0,0.0,31.0,0,0,0,0,0,0,...,0.62069,1606.939091,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0
3,A00008,0,0.0,22.0,0,0,0,0,0,0,...,0.00000,1570.506667,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.5
4,A00009,0,0.0,31.0,0,0,0,0,0,0,...,-999.00000,-999.000000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [37]:
no_features = ['y','user_id']
features = [col for col in df_train.columns if col not in no_features]


In [38]:
ycol = 'y'
feature_names = features
model = cbt.CatBoostClassifier(eval_metric='AUC',
                               max_depth=6,
                               learning_rate=0.05,
                               n_estimators=10000,
                               reg_lambda=12,
                               #                                task_type='GPU',
                               random_state=seed)


df_oof = []
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train[feature_names],df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(
        fold_id + 1))
    # print(df_oof.loc[val_idx]['prob'])
    cbt_model = model.fit(X_train,
                          Y_train,
                        #   eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                        #    silent=True, 
                           use_best_model=True,
                          verbose=100,
                          early_stopping_rounds=50)

    pred_val = cbt_model.predict_proba(
        X_val)[:, 1]
   
    df_oof = df_train.iloc[val_idx][[
        'user_id', ycol]].copy()
    df_oof['pred'] = pred_val
    df_oof.append(df_oof)
    

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': cbt_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del cbt_model,  X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

0:	test: 0.7234000	test1: 0.7236738	best: 0.7236738 (0)	total: 358ms	remaining: 59m 44s
100:	test: 0.8127978	test1: 0.7828721	best: 0.7828721 (100)	total: 31.1s	remaining: 50m 53s
200:	test: 0.8342791	test1: 0.7874861	best: 0.7878151 (189)	total: 1m 1s	remaining: 49m 50s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7878150957
bestIteration = 189

Shrink model to first 190 iterations.

Fold_2 Training ================================

0:	test: 0.7245725	test1: 0.7370380	best: 0.7370380 (0)	total: 299ms	remaining: 49m 48s
100:	test: 0.8121150	test1: 0.7814630	best: 0.7814630 (100)	total: 31.1s	remaining: 50m 46s
200:	test: 0.8331699	test1: 0.7861722	best: 0.7861880 (197)	total: 1m	remaining: 49m 32s
300:	test: 0.8534552	test1: 0.7865839	best: 0.7866575 (298)	total: 1m 31s	remaining: 48m 53s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7867927378
bestIteration = 314

Shrink model to first 315 

In [39]:
df_oof.to_csv('cbt_oof_feature_engineering.csv',index=False)

In [40]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()


In [67]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
Threshold = 0.35

In [68]:
accuracy_score(df_oof['y'], df_oof['pred'] >Threshold)

0.7892824704813806

In [69]:
recall_score(df_oof['y'], df_oof['pred'] >Threshold,average='macro')

0.668014887945295

In [70]:
f1_score(df_oof['y'], df_oof['pred'] >Threshold,average='macro')


0.6665278461363722

In [71]:
precision_score(df_oof['y'], df_oof['pred'] >Threshold,average='macro')

0.6651096511664026

In [72]:
def get_roc_auc(y_test,y_score):
    # y_score =clf.predict_proba(test_vecs)[:,1]
    auc_s = roc_auc_score(y_test,y_score)   
    # print(y_predict)
    # print(y_score)
#     print(cm)
    print(auc_s)
    # Compute ROC curve and ROC area for each class
    fpr,tpr,threshold = roc_curve(y_test, y_score) ###计算真正率和假正率
    roc_auc = auc(fpr,tpr) ###计算auc的值
    
    plt.figure()
    lw = 2
    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % roc_auc) ###假正率为横坐标，真正率为纵坐标做曲线
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [73]:
get_roc_auc(df_oof['y'], df_oof['pred'])

0.7815980307452403


<Figure size 432x288 with 0 Axes>

<Figure size 720x720 with 1 Axes>

In [74]:


from sklearn.metrics import precision_recall_curve
# from sklearn.utils.fixes import signature
plt.figure("P-R Curve")
plt.title('Precision/Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
precision, recall, thresholds = precision_recall_curve(df_oof['y'], df_oof['pred'])
#print(precision)
#print(recall)
#print(thresholds)
plt.plot(recall,precision)
plt.show()

<Figure size 432x288 with 1 Axes>

In [75]:
fpr,tpr,thresholds= roc_curve(df_oof['y'], df_oof['pred'])
print ('KS:',max(tpr-fpr))

KS: 0.44351167960888843


In [76]:
def get_confusion_matrix(test_y,pred):
    f,ax=plt.subplots()

    C2= confusion_matrix(test_y, pred, labels=[0, 1])
    print(C2) #打印出来看看
    sns.set(font_scale=1.5)
    sns.heatmap(C2,annot=True,ax=ax, fmt='.20g',cmap="YlGnBu") #画热力图

    ax.set_title('confusion matrix') #标题
    ax.set_xlabel('predict') #x轴
    ax.set_ylabel('true') #y轴

In [77]:
get_confusion_matrix(df_oof['y'], df_oof['pred'] > 0.364)

[[1563  211]
 [ 238  190]]


<Figure size 432x288 with 2 Axes>

In [78]:
max_f1 = 0
Threshold = 0
for i in range(200,400):
    f1 = f1_score(df_oof['y'], df_oof['pred'] >i * 1e-3,average='macro')
    # f1 = f1_score(df_oof['y'].astype('int64'), (df_oof['pred'] >=i * 1e-3).astype('int64'),average='weighted')
    if max_f1 < f1:
        max_f1 = f1
        Threshold =  i

max_f1 ,Threshold * 1e-3

(0.6689083401072565, 0.356)